In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import gridspec
from scipy import stats

from one.api import ONE
from brainbox.io.one import SessionLoader

from iblnm.config import EVENT2COLOR
from iblnm.util import protocol2type, _load_event_times
from iblnm.resp import get_responses

 ## Pick a session using df_sessions from a previous query

In [4]:
df_sessions = pd.read_parquet('metadata/sessions.pqt').query('remote_photometry == True')
df_sessions = df_sessions[df_sessions['target'].apply(len) > 0]
df_sessions['target_NM'] = df_sessions.apply(lambda x: '-'.join([x['target'][0], x['NM']]), axis='columns')
df_sessions['target_NM'].value_counts()

target_NM
NBM-ACh      255
VTA-DA       169
LC-NE        168
SI-ACh        42
SNc-DA        13
SNC-DA        13
NBM-l-ACh      8
PPT-ACh        6
VTA-none       2
Name: count, dtype: int64

In [5]:
# Pick a target
target_NM = 'VTA-DA'
df_target = df_sessions.query('target_NM == @target_NM')
df_target.groupby(['subject', 'session_type']).count()['session_n']

subject    session_type
ZFM-08488  training         7
ZFM-08554  training         7
ZFM-08689  habituation      3
           training         5
ZFM-08818  habituation      3
           training        47
ZFM-08827  habituation      3
           training        48
ZFM-08828  habituation      3
           misc             1
           training        25
ZFM-09044  habituation      3
           training        14
Name: session_n, dtype: int64

In [6]:
# Pick a session
subject = 'ZFM-08827'
session_type = 'training'
session_n = -1
session = df_target.query('(subject == @subject) & (session_type == @session_type)').iloc[session_n]
eid = session['eid']
print(session[['subject', 'target_NM', 'session_type', 'session_n', 'start_time', 'eid']])

# Get responses to task events
session = _load_event_times(session, one)

subject                                    ZFM-08827
target_NM                                     VTA-DA
session_type                                training
session_n                                        4.0
start_time                2025-05-20T13:23:20.053276
eid             444b1cf1-eca0-44da-964e-3b2af954a82d
Name: 554, dtype: object


(S3) /home/davide/Downloads/ONE/alyx.internationalbrainlab.org/mainenlab/Subjects/ZFM-08827/2025-05-20/001/alf/task_00/_ibl_trials.table.pqt


## Pick a sessions by querying Alyx

In [3]:
one = ONE()

In [27]:
# Pick a subject
subject = 'ZFM-08827'

df_sessions = pd.DataFrame(one.alyx.rest('sessions', 'list', project='ibl_fibrephotometry', subject=subject, dataset='photometry.signal.pqt')).rename(columns={'id':'eid'})
df_sessions.head()

,eid,subject,start_time,number,lab,projects,url,task_protocol
0,9c442db0-6b0c-4745-b34c-57ad2d5de9f3,ZFM-08827,2025-09-01T14:55:20.672877,1,mainenlab,[ibl_fibrephotometry],https://alyx.internationalbrainlab.org/session...,_iblrig_tasks_trainingChoiceWorld8.29.0
1,6171e7fc-4d4d-43db-a787-9678f1b96087,ZFM-08827,2025-08-29T14:34:17.920412,1,mainenlab,[ibl_fibrephotometry],https://alyx.internationalbrainlab.org/session...,_iblrig_tasks_trainingChoiceWorld8.29.0
2,b907588e-eae8-4231-a6ac-446f5815d182,ZFM-08827,2025-08-28T14:25:18.822169,1,mainenlab,[ibl_fibrephotometry],https://alyx.internationalbrainlab.org/session...,_iblrig_tasks_trainingChoiceWorld8.29.0
3,fa2af6db-0587-4cbe-8d1f-cb62d60734af,ZFM-08827,2025-08-27T13:18:08.698924,1,mainenlab,[ibl_fibrephotometry],https://alyx.internationalbrainlab.org/session...,_iblrig_tasks_trainingChoiceWorld8.29.0
4,ebcca2c8-9f10-45a7-acd0-c4a9808343e3,ZFM-08827,2025-08-26T18:10:09.943452,1,mainenlab,[ibl_fibrephotometry],https://alyx.internationalbrainlab.org/session...,_iblrig_tasks_trainingChoiceWorld8.29.0


In [33]:
# Pick a session
session_i = 1
session = df_sessions.iloc[session_i]
session

eid                           6171e7fc-4d4d-43db-a787-9678f1b96087
subject                                                  ZFM-08827
start_time                              2025-08-29T14:34:17.920412
number                                                           1
lab                                                      mainenlab
projects                                     [ibl_fibrephotometry]
url              https://alyx.internationalbrainlab.org/session...
task_protocol              _iblrig_tasks_trainingChoiceWorld8.29.0
Name: 1, dtype: object

In [35]:
# Get responses to task events
session = _load_event_times(session, one)

In [38]:
# Get photometry data
signals = one.load_dataset(id=session['eid'], dataset='photometry.signal.pqt')
locations = one.load_dataset(id=session['eid'], dataset='photometryROI.locations.pqt').reset_index()
photometry = signals[locations['ROI'].to_list() + ['name']].set_index(signals['times']).dropna()
print(locations)

  ROI      fiber brain_region
0  G2  fiber_VTA          VTA


In [39]:
# Pick an ROI
# roi = 'G0'
roi = locations['ROI'].iloc[0]

In [44]:
# Restrict raw photometry to the task period
buffer = 0  # seconds
loader = SessionLoader(one, eid=eid)
## FIXME: appropriately handle cases with multiple task collections
loader.load_trials(collection='alf/task_00')
timings = [col for col in loader.trials.columns if col.endswith('_times')]
t0 = loader.trials[timings].min().min()
t1 = loader.trials[timings].max().max()
i0 = photometry.index.searchsorted(t0 - buffer)
i1 = photometry.index.searchsorted(t1 + buffer)
photometry = photometry.iloc[i0:i1].copy()

# Pull channels out of df
gcamp = photometry.query('name == "GCaMP"')
iso = photometry.query('name == "Isosbestic"')

In [45]:
events = ['cue', 'movement', 'reward', 'omission']
psths = []
for event in events:
    responses, tpts = get_responses(gcamp[roi], session[f'{event}_times'])
    psths.append(responses)

In [46]:
%matplotlib qt

fig = plt.figure(figsize=(12, 8))
grid = gridspec.GridSpec(2, 3)

ax2 = fig.add_subplot(grid[1, 0])
ax3 = fig.add_subplot(grid[1, 1])
ax4 = fig.add_subplot(grid[1, 2])
ax1 = fig.add_subplot(grid[0, :])

ax1.plot(gcamp[roi])
ax1.plot(iso[roi], color='gray')
ax1.set_ylabel('Signal (a.u.)')
ax1.set_title(f"{session['subject']} {session['start_time']} {session['task_protocol']}")

colors = [EVENT2COLOR[event] for event in events]
for event, responses, color, ax in zip(events, psths, colors, [ax2, ax3, ax4, ax4]):
    ax.plot(tpts, responses.mean(axis=0), color=color, label=event)
    ax.plot(tpts, responses.mean(axis=0) - stats.sem(responses, axis=0), ls='--', color=color)
    ax.plot(tpts, responses.mean(axis=0) + stats.sem(responses, axis=0), ls='--', color=color)
    ax.axvline(0, ls='--', color='black', alpha=0.5)
    ax.axhline(0, ls='--', color='gray', alpha=0.5)
    ax.set_xlabel('Time (s)')
    if event != 'omission':
        ax.set_title(event.capitalize())
    ax.ticklabel_format(axis='y', style='sci', scilimits=[-2, 2])